# Lihtne pildiklassifikaator

See märkmik näitab, kuidas klassifitseerida pilte eelnevalt treenitud närvivõrgu abil.

**Mida õpid:**
- Kuidas laadida ja kasutada eelnevalt treenitud mudelit
- Piltide eeltöötlus
- Ennustuste tegemine piltide põhjal
- Usaldusväärsuse skooride mõistmine

**Kasutusjuht:** Objektide tuvastamine piltidel (näiteks "kass", "koer", "auto" jne)

---


## Samm 1: Vajalike teekide importimine

Importime vajalikud tööriistad. Ära muretse, kui sa veel kõigest aru ei saa!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Samm 2: Eeltreenitud mudeli laadimine

Kasutame **MobileNetV2**, närvivõrku, mis on juba treenitud miljonite piltide peal.

Seda nimetatakse **ülekandeõppeks** - kasutame mudelit, mille keegi teine on juba treeninud!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Samm 3: Abifunktsioonid

Loome funktsioonid piltide laadimiseks ja ettevalmistamiseks meie mudeli jaoks.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Samm 4: Testimine näidispiltidega

Proovime klassifitseerida mõned internetist leitud pildid!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klassifitseeri iga pilt


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Samm 5: Proovi oma pilte!

Asenda allolev URL mis tahes pildi URL-iga, mida soovid klassifitseerida.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Mis Just Juhtus?

1. **Laadisime eelnevalt treenitud mudeli** - MobileNetV2 on treenitud miljonite piltidega
2. **Eeltöötlesime pilte** - Muutsime nende suurust ja vormindasime mudeli jaoks
3. **Mudel tegi ennustusi** - Väljundiks olid tõenäosused 1000 objekti klassi kohta
4. **Dekodeerisime tulemused** - Muutsime numbrid inimloetavateks siltideks

### Usaldusväärsuse skooride mõistmine

- **90-100%**: Väga kindel (peaaegu kindlasti õige)
- **70-90%**: Kindel (tõenäoliselt õige)
- **50-70%**: Mõnevõrra kindel (võib olla õige)
- **Alla 50%**: Vähe kindel (ebakindel)

### Miks ennustused võivad olla valed?

- **Ebatavaline nurk või valgustus** - Mudel on treenitud tüüpiliste fotode põhjal
- **Mitu objekti** - Mudel eeldab ühte peamist objekti
- **Haruldased objektid** - Mudel tunneb ainult 1000 kategooriat
- **Madala kvaliteediga pilt** - Udused või pikslilised pildid on raskemad


## 🚀 Järgmised sammud

1. **Proovi erinevaid pilte:**
   - Leia pilte [Unsplashist](https://unsplash.com)
   - Paremklõps → "Kopeeri pildi aadress", et saada URL

2. **Katseta:**
   - Mis juhtub abstraktse kunstiga?
   - Kas see suudab objekte ära tunda erinevatest nurkadest?
   - Kuidas see käsitleb mitut objekti korraga?

3. **Õpi rohkem:**
   - Uuri [Arvutinägemise õppetunde](../lessons/4-ComputerVision/README.md)
   - Õpi, kuidas treenida oma pildiklassifikaatorit
   - Saa aru, kuidas CNN-id (konvolutsioonilised närvivõrgud) töötavad

---

## 🎉 Palju õnne!

Sa just ehitasid pildiklassifikaatori, kasutades tipptasemel närvivõrku!

Sama tehnika on aluseks:
- Google Photos (sinu fotode organiseerimine)
- Isejuhtivad autod (objektide tuvastamine)
- Meditsiiniline diagnoos (röntgenpiltide analüüs)
- Kvaliteedikontroll (vigade tuvastamine)

Jätka avastamist ja õppimist! 🚀



---

**Lahtiütlus**:  
See dokument on tõlgitud, kasutades AI tõlketeenust [Co-op Translator](https://github.com/Azure/co-op-translator). Kuigi püüame tagada täpsust, palun arvestage, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algkeeles tuleks lugeda autoriteetseks allikaks. Olulise teabe puhul on soovitatav kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valede tõlgenduste eest.
